In [1]:
# Importing Packages and Libraries
import shap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [2]:
# Load data
file_path1 = '/Users/paulahofmann/Documents/Coding/Online-Review/FeaturePreperation/Data_with_Features/Final Data/Hedonic_Final.csv'
df = pd.read_csv(file_path1)



In [3]:
# Drop unnecessary columns
columns_to_drop = ['title_x', 'text', 'images', 'asin', 'parent_asin', 'user_id',
       'timestamp', 'verified_purchase', 'text_cleaned',
       'text_cleaned1','main_category','features','neutral_score','day_of_week',]
df.drop(columns=columns_to_drop, inplace=True)

In [4]:
# Mapping dictionary for sentiment transformation
sentiment_mapping = {'positive': 2, 'neutral': 1, 'negative': 0}

# Using map function to create a new column 'sentiment_c' with transformed values
df['sentiment_c'] = df['Sentiment_Classification'].map(sentiment_mapping)

In [5]:
# Create a new column 'helpful' based on the condition
df['helpful'] = (df['helpful_vote'] > 0).astype(int)


In [6]:
# Define features and target variable
features = ['rating', 'sentiment_c','word_count', 'sent_count', 'sent_length', 'title_length', 'elap_days',
       'image', 'ver_purch','FRE','#nouns','#adj', '#adv', 'subjective_score','product']
target = 'helpful'

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
import statsmodels.api as sm

# Assuming df is your DataFrame and features & target are defined appropriately
# Example:
# df = pd.read_csv('your_data.csv')

# Create dummy variables for 'product' column
#product_dummies = pd.get_dummies(df['product'], drop_first=True,dtype=int)

# Define numerical features and the target
numerical_features = ['rating', 'sentiment_c', 'word_count', 'sent_count', 'sent_length', 'title_length', 'elap_days', 'image', 'ver_purch', 'FRE', '#nouns', '#adj', '#adv', 'subjective_score']
target = 'helpful'

# Concatenate numerical features and dummy variables
#X = pd.concat([df[numerical_features], product_dummies], axis=1)
X = df [numerical_features]
y = df[target]



In [8]:
import statsmodels.api as sm
logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.398471
         Iterations 7
                         Results: Logit
Model:              Logit            Method:           MLE       
Dependent Variable: helpful          Pseudo R-squared: 0.120     
Date:               2024-05-28 19:00 AIC:              17728.8570
No. Observations:   22211            BIC:              17840.9738
Df Model:           13               Log-Likelihood:   -8850.4   
Df Residuals:       22197            LL-Null:          -10057.   
Converged:          1.0000           LLR p-value:      0.0000    
No. Iterations:     7.0000           Scale:            1.0000    
-----------------------------------------------------------------
                  Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
-----------------------------------------------------------------
rating           -0.3167   0.0197 -16.0821 0.0000 -0.3553 -0.2781
sentiment_c       0.0551   0.0361   1.5245 0.1274 -0.0157  0.1260


In [9]:
# Deleting Sentiment Classification due not significant
numerical_features = ['rating', 'word_count', 'sent_count', 'sent_length', 'title_length', 'elap_days', 'image', 'ver_purch', 'FRE', '#nouns', '#adj', '#adv', 'subjective_score']
target = 'helpful'

# Concatenate numerical features and dummy variables
#X = pd.concat([df[numerical_features], product_dummies], axis=1)
X = df [numerical_features]
y = df[target]

import statsmodels.api as sm
logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.398523
         Iterations 7
                         Results: Logit
Model:              Logit            Method:           MLE       
Dependent Variable: helpful          Pseudo R-squared: 0.120     
Date:               2024-05-28 19:00 AIC:              17729.1944
No. Observations:   22211            BIC:              17833.3029
Df Model:           12               Log-Likelihood:   -8851.6   
Df Residuals:       22198            LL-Null:          -10057.   
Converged:          1.0000           LLR p-value:      0.0000    
No. Iterations:     7.0000           Scale:            1.0000    
-----------------------------------------------------------------
                  Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
-----------------------------------------------------------------
rating           -0.2943   0.0131 -22.4353 0.0000 -0.3200 -0.2686
word_count        0.0050   0.0014   3.6845 0.0002  0.0024  0.0077


In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

# Split data for test and training data
X_train, X_test, y_train, y_test = train_test_split(
    df[numerical_features],  
    df[target], 
    test_size=0.2,  
    random_state=42  # Set a random seed for reproducibility
)

In [11]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

pd.DataFrame(X_train, columns=X.columns)


,rating,word_count,sent_count,sent_length,title_length,elap_days,image,ver_purch,FRE,#nouns,#adj,#adv,subjective_score
0,0.535690,-0.171701,0.269534,-0.520649,-0.315217,2.076454,-0.194396,0.278743,1.097237,-0.796140,-0.639321,-0.249450,0.713360
1,-0.255050,-0.615166,-0.649940,-1.034781,-0.315217,0.013387,-0.194396,0.278743,-1.351391,1.230839,4.979213,-0.869362,0.720624
2,0.535690,-0.276046,-0.190203,-0.322906,-0.315217,0.394687,-0.194396,0.278743,-1.174777,-1.664845,-0.525432,1.455309,-0.016021
3,-0.255050,0.271764,-0.190203,0.922876,-0.315217,0.567231,-0.194396,0.278743,0.032865,-0.490919,-0.602384,0.135901,0.054558
4,0.535690,-0.458649,-0.649940,-0.085614,-0.315217,0.508651,-0.194396,0.278743,0.094342,-0.699617,0.929819,-0.869362,-0.034329
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17763,-2.627272,-0.458649,-0.190203,-0.738166,-0.315217,1.491723,-0.194396,0.278743,0.773185,-0.699617,0.929819,0.508221,-0.663855
17764,0.535690,-0.458649,-0.649940,-0.085614,-0.315217,0.110310,-0.194396,0.278743,0.094342,0.265611,-1.094878,0.508221,-2.264112
17765,0.535690,-0.354304,-0.190203,-0.500875,-0.315217,1.405451,-0.194396,0.278743,0.323575,-0.996610,0.306836,1.038060,0.620752
17766,0.535690,-0.197787,0.729272,-0.708505,-0.315217,-0.397735,-0.194396,0.278743,0.237091,0.621222,1.302790,-0.869362,0.383323


In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report

# Initialize the logistic regression model
logreg_model = LogisticRegression(solver='lbfgs', max_iter=1000)  # You can adjust max_iter as needed

# Train the model
logreg_model.fit(X_train, y_train)

# Predict on the testing set
y_pred = logreg_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate F1 score
f1 = f1_score(y_test, y_pred)
print("F1 Score:", f1)

# Calculate AUC
auc = roc_auc_score(y_test, y_pred)
print("AUC:", auc)

# Generate classification report
print(classification_report(y_test, y_pred))

from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

Accuracy: 0.8370470402880936
F1 Score: 0.2113289760348584
AUC: 0.5551949384035673
              precision    recall  f1-score   support

           0       0.85      0.98      0.91      3690
           1       0.59      0.13      0.21       753

    accuracy                           0.84      4443
   macro avg       0.72      0.56      0.56      4443
weighted avg       0.80      0.84      0.79      4443

[[3622   68]
 [ 656   97]]


In [14]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

NameError: name 'logreg' is not defined

In [15]:
import statsmodels.api as sm
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report

# Adding a constant to the model (intercept term)
X_train_const = sm.add_constant(X_train)
X_test_const = sm.add_constant(X_test)

# Initialize the logistic regression model
logit_model = sm.Logit(y_train, X_train_const)

# Train the model
result = logit_model.fit()

# Print the summary of the model
print(result.summary())

# Get p-values of coefficients
p_values = result.pvalues

# Significant coefficients (assuming alpha = 0.05)
significant_coefs = p_values[p_values < 0.05]
print("Significant coefficients:")
print(significant_coefs)

# Predict on the testing set
y_pred_prob = result.predict(X_test_const)
y_pred = (y_pred_prob >= 0.5).astype(int)  # Convert probabilities to binary outcomes

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate F1 score
f1 = f1_score(y_test, y_pred)
print("F1 Score:", f1)

# Calculate AUC
auc = roc_auc_score(y_test, y_pred_prob)
print("AUC:", auc)

# Generate classification report
print(classification_report(y_test, y_pred))


Optimization terminated successfully.
         Current function value: 0.394377
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                helpful   No. Observations:                17768
Model:                          Logit   Df Residuals:                    17754
Method:                           MLE   Df Model:                           13
Date:                Tue, 28 May 2024   Pseudo R-squ.:                  0.1280
Time:                        19:01:16   Log-Likelihood:                -7007.3
converged:                       True   LL-Null:                       -8035.5
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.8133      0.024    -76.761      0.000      -1.860      -1.767
x1            -0.3281      0.